| 단계               | 역할                                |
| ---------------- | --------------------------------- |
| `yolov3.cfg`     | 네트워크의 층 구조(Backbone, Detector) 정의 |
| `yolov3.weights` | 구조에 들어갈 사전학습된 파라미터                |
| OpenCV `readNet` | 두 파일을 로드해 실행 가능한 모델로 조립           |

| 단계    | 설명                                           |
| ----- | -------------------------------------------- |
| **1** | COCO 클래스 이름 읽기 (`coco_names.txt`)            |
| **2** | YOLO 모델 로드 (`yolov3.weights` + `yolov3.cfg`) |
| **3** | 입력 이미지 전처리 (blob 생성)                         |
| **4** | 모델에 blob 입력하고 forward로 출력 레이어 실행             |
| **5** | 신뢰도 50% 이상만 박스/클래스 추출                        |
| **6** | NMS로 겹치는 박스 제거                               |
| **7** | 최종 박스와 클래스 이름/신뢰도를 영상에 출력                    |
| **8** | OpenCV로 결과 이미지 시각화                           |

In [ ]:
import numpy as np
import cv2 as cv
import sys

# [1] YOLOv3 모델과 클래스 이름 구성 함수
def construct_yolo_v3():
    # coco_names.txt 파일에서 객체 클래스 이름(예: person, car, dog ...) 읽기
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]  # 각 줄의 개행문자 제거

    # YOLO 모델 로드 (가중치와 설정파일)
    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')

    # 모델의 모든 레이어 이름 가져오기
    layer_names = model.getLayerNames()

    # 출력 레이어만 선택 (YOLOv3 구조에서 탐지 결과가 나오는 레이어)
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]

    return model, out_layers, class_names

# [2] YOLO로 이미지에서 객체 탐지하는 함수
def yolo_detect(img, yolo_model, out_layers):
    # 이미지 높이와 너비 가져오기
    height, width = img.shape[0], img.shape[1]

    # 입력 이미지 전처리: blob 만들기
    # scale factor 1/256, 크기 448x448으로 resize, BGR->RGB 변환
    test_img = cv.dnn.blobFromImage(
        img,         # 원본 이미지
        1.0/256,     # 스케일 팩터 (픽셀값을 0~1로)
        (448,448),   # 크기 리사이즈
        (0,0,0),     # Mean 값 (평균값 빼기)
        swapRB=True  # BGR → RGB 변환
    )

    # YOLO 모델에 blob 입력
    yolo_model.setInput(test_img)

    # forward()로 출력 레이어 실행해서 탐지 결과 받기
    output3 = yolo_model.forward(out_layers)

    # 박스, 신뢰도, 클래스 ID 리스트 초기화
    box, conf, id = [], [], []

    # output3에는 여러 레이어별 탐지 결과가 들어있음
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]            # 앞 5개는 위치, 뒤는 클래스별 점수
            class_id = np.argmax(scores)  # 가장 높은 클래스 선택
            confidence = scores[class_id] # 그 클래스의 신뢰도
            if confidence > 0.5:          # 신뢰도 50% 이상인 것만 사용
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)  # 중심 좌표
                w, h = int(vec85[2]*width), int(vec85[3]*height)              # 박스 너비, 높이
                x, y = int(centerx - w/2), int(centery - h/2)                  # 좌상단 좌표로 변환
                box.append([x, y, x+w, y+h])      # [x1,y1,x2,y2]
                conf.append(float(confidence))    # 신뢰도 저장
                id.append(class_id)               # 클래스 ID 저장

    # NMS (Non-Maximum Suppression)로 겹치는 박스 제거
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)

    # 최종 물체 리스트: 박스 좌표 + 신뢰도 + 클래스ID
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]

    return objects

# [3] YOLOv3 모델 생성
model, out_layers, class_names = construct_yolo_v3()

# [4] 클래스별 랜덤 색상 생성 (시각화용)
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

# [5] 이미지 읽기
img = cv.imread('soccer.jpg')
if img is None:
    sys.exit('파일이 없습니다.')

# [6] YOLO 탐지 수행
res = yolo_detect(img, model, out_layers)

# [7] 탐지된 물체 박스와 텍스트 출력
for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + ' %.3f' % confidence  # 클래스 이름과 신뢰도
    cv.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)  # 박스 그리기
    cv.putText(img, text, (x1, y1 + 30), cv.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)

# [8] 결과 이미지 표시
cv.imshow("Object detection by YOLO v.3", img)
cv.waitKey()
cv.destroyAllWindows()

cv.dnn.blobFromImage: 이미지 전처리 (크기/스케일링/BGR→RGB 변환) \
cv.dnn.readNet: DNN 모델 로드 \
model.forward(out_layers): YOLO의 출력 레이어만 실행 \
NMS (cv.dnn.NMSBoxes): 박스 겹침 제거 알고리즘 \
YOLO output 구조: [x, y, w, h, obj_conf, class_scores...] \
bounding box 좌표: YOLO는 중심 좌표 + 크기 → 좌상단/우하단으로 변환 필요